In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
import numpy as np
import math
import random

In [4]:
import time

In [5]:
def show_state(env, episode,step, info=""):
    plt.figure(99999,figsize=[8,6])
    plt.clf()
    plt.imshow(env.render(mode='rgb_array'))
    plt.title("episode: {} step: {} ".format(episode,step))
    #plt.title("%s | Step: %d %s" % (env._spec.id,step, info))
    plt.axis('off')

    display.clear_output(wait=True)
    display.display(plt.gcf())

In [6]:
import gym

In [7]:
use_cuda = torch.cuda.is_available()
device=torch.device("cuda:4" if use_cuda else "cpu")

In [8]:
###### PARAMS ######
learning_rate = 0.01
num_episodes = 50000
gamma=0.999999
#gamma=0.85
egreedy = 0.9
egreedy_final = 0.01
egreedy_decay = 500

report_interval=10

score_to_solve = 195

hidden_layer_size=64

replay_memory_size=50000

batch_size=32

update_target_frequency = 100

clip_error=False

In [9]:
env = gym.make('Breakout-ramDeterministic-v4')

In [10]:
#env = gym.wrappers.Monitor(env, '../mp4/sandbox10',video_callable=lambda episode_id: True,force=True)
env = gym.wrappers.Monitor(env, '../mp4/breakout_DQN',video_callable=lambda episode_id: episode_id%100==0,force=True)

In [11]:
number_of_inputs=env.observation_space.shape[0]
number_of_outputs=env.action_space.n

In [12]:
def calculate_epsilon(steps_done):
    epsilon = egreedy_final + (egreedy - egreedy_final) * \
              math.exp(-1. * steps_done / egreedy_decay )
    return epsilon

In [13]:
class ExperienceReplay():
    def __init__(self, capacity):
        self.capacity=capacity
        self.memory=[]
        self.position=0
        
    
    def push(self, state,
             action, new_state,
             reward, done):
        
            transition=(state,action,new_state,reward,done)
            
            if self.position>=len(self.memory):
                self.memory.append(transition)
            else:
                self.memory[self.position]=transition
                
            self.position=(self.position+1)%self.capacity
        
    
    def sample(self,batch_size):
        return zip(*random.sample(self.memory, batch_size))
    
    def __len__(self):
        return len(self.memory)
    

In [14]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear1 = nn.Linear(number_of_inputs,hidden_layer_size)
        self.linear2 = nn.Linear(hidden_layer_size,number_of_outputs)
        
        self.activation=nn.Tanh()
        
    def forward(self, x):
        output1 = self.linear1(x)
        output1 = self.activation(output1)
        output2 = self.linear2(output1)
        
        return output2

In [15]:
class QNet_Agent():
    def __init__(self):
        
        self.nn = NeuralNetwork().to(device)
        self.target_nn = NeuralNetwork().to(device)
        
        self.loss_function = nn.MSELoss()
        self.optimizer = optim.Adam(params=self.nn.parameters(), lr=learning_rate)
        
        self.update_target_counter = 0
        
    def select_action(self,state,epsilon):
        
        random_for_egreedy=torch.rand(1).item()
        
        if random_for_egreedy>epsilon:
            self.nn.eval()
            with torch.no_grad():
                state=torch.Tensor(state).to(device)
                predicted_value_from_nn=self.nn(state)
                action=torch.argmax(predicted_value_from_nn).item()
        else:
            action=env.action_space.sample()
                
                
        return action
    
    def optimize(self):
        
        if len(memory)<batch_size:
            return
        
        state, action, new_state, reward, done = memory.sample(batch_size)
        
        state=torch.Tensor(state).to(device)
        new_state=torch.Tensor(new_state).to(device)
        reward=torch.Tensor(reward).to(device)
        
        #the view call below is to transform into column vector
        #so that it can be used in the gather call
        #i.e. we will use it to pick out from the computed value
        #tensor only values indexed by selected action
        action=(torch.Tensor(action).view(-1,1).long()).to(device)
        #print('action: ')
        #print(action)
        #print('contiguous?', action.is_contiguous())
        done=torch.Tensor(done).to(device)
        
        #print('shape of: state, new state, reward, action, done:')
        #print(state.shape)
        #print(new_state.shape)
        #print(reward.shape)
        #print(action.shape)
        #print(done.shape)
        
        
        self.nn.eval()
        self.target_nn.eval()
            
        new_state_values=self.target_nn(new_state).detach()
        #print('shape of: new_state_values')
        #print(new_state_values.shape)
        
        max_new_state_values=torch.max(new_state_values,dim=1)[0]
        #print('shape of: max_new_state_values')
        #print(max_new_state_values.shape)
        target_value=(reward + (1-done)*gamma*max_new_state_values).view(-1,1)
        
        #print('shape of: target_value')
        #print(target_value.shape)
        self.nn.train()
        
        #this will select only the values of the desired actions
        predicted_value=torch.gather(self.nn(state),1,action)
        #print('shape of: predicted_value')
        #print(predicted_value.shape)
        
        
        loss=self.loss_function(predicted_value,target_value)
        self.optimizer.zero_grad()
        loss.backward()
        
        if clip_error:
            for param in self.nn.parameters():
                param.grad.clamp_(-1.0,1.0)
        
        self.optimizer.step()
        
        if self.update_target_counter % update_target_frequency == 0:
            #print("***********************")
            #print("UPDATING TARGET NETWORK")
            #print("update counter: {}".format(self.update_target_counter))
            #print("***********************")
            self.target_nn.load_state_dict(self.nn.state_dict())
        
        self.update_target_counter+=1
        

In [16]:
a=torch.randn([3,3])
print(a)

tensor([[ 1.5225,  1.1023,  1.4868],
        [-2.0261, -0.6412,  0.4389],
        [ 1.6641,  0.0969,  0.9602]])


In [17]:
a.clamp_(-1.0,1.0)

tensor([[ 1.0000,  1.0000,  1.0000],
        [-1.0000, -0.6412,  0.4389],
        [ 1.0000,  0.0969,  0.9602]])

In [18]:
a

tensor([[ 1.0000,  1.0000,  1.0000],
        [-1.0000, -0.6412,  0.4389],
        [ 1.0000,  0.0969,  0.9602]])

In [19]:
memory=ExperienceReplay(replay_memory_size)

In [20]:
qnet_agent=QNet_Agent()

In [21]:
seed_value=23
env.seed(seed_value)
torch.manual_seed(seed_value)
random.seed(seed_value)

In [ ]:
%%time

steps_total=np.full([num_episodes],-999,dtype=np.int32)
reward_total=np.full([num_episodes],-999,dtype=np.int32)

frames_total=0

solved_after = 0
solved = False

start_time = time.time()

for i_episode in range(num_episodes):
    
    state = env.reset()
    #for step in range(100):
    step=0
    reward_total[i_episode]=0
    
    while True:
        
        step+=1
        frames_total += 1
        
        epsilon=calculate_epsilon(frames_total)
        
        #action=env.action_space.sample()
        action=qnet_agent.select_action(state,epsilon)
        
        new_state, reward, done, info = env.step(action)
        memory.push(state, action, new_state,
                     reward, done)
        
        reward_total[i_episode]+=reward
        
        qnet_agent.optimize()
        
        state=new_state
        
        if done:
            steps_total[i_episode]=step
            
            if i_episode>100:
                mean_reward_100 = np.sum(reward_total[i_episode-100:i_episode])/100
            
                if (mean_reward_100 > score_to_solve and solved == False):
                    print("SOLVED! After %i episodes " % i_episode)
                    solved_after = i_episode
                    solved = True
            
            if (i_episode % report_interval == 0 and i_episode>1):
                print("**** Episode  {} **** ".format(i_episode))
                recent_avg_reward=np.average(reward_total[i_episode-report_interval:i_episode])
                print("Recent average reward: {}".format(recent_avg_reward))
                if i_episode>100:
                    print("Reward over last 100: {}".format(mean_reward_100))
                full_avg_so_far=np.average(reward_total[:i_episode])
                print("Average over all episodes so far: {}".format(full_avg_so_far))
                print("epsilon: {}".format(epsilon))
            
                #print("Episode {} finished after: {}".format(i_episode,step))
            break
            
if solved:
    print("Solved after %i episodes" % solved_after)
        

**** Episode  10 **** 
Recent average reward: 1.9
Average over all episodes so far: 1.9
epsilon: 0.016405912952911657
**** Episode  20 **** 
Recent average reward: 2.1
Average over all episodes so far: 2.0
epsilon: 0.010057683821353419
**** Episode  30 **** 
Recent average reward: 1.4
Average over all episodes so far: 1.8
epsilon: 0.010001167630803692
**** Episode  40 **** 
Recent average reward: 1.1
Average over all episodes so far: 1.625
epsilon: 0.0100000226176827
**** Episode  50 **** 
Recent average reward: 2.0
Average over all episodes so far: 1.7
epsilon: 0.010000000258394452
**** Episode  60 **** 
Recent average reward: 1.2
Average over all episodes so far: 1.6166666666666667
epsilon: 0.010000000004248166
**** Episode  70 **** 
Recent average reward: 2.3
Average over all episodes so far: 1.7142857142857142
epsilon: 0.010000000000030334
**** Episode  80 **** 
Recent average reward: 1.7
Average over all episodes so far: 1.7125
epsilon: 0.010000000000000387
**** Episode  90 **** 


**** Episode  600 **** 
Recent average reward: 1.7
Reward over last 100: 2.27
Average over all episodes so far: 1.955
epsilon: 0.01
**** Episode  610 **** 
Recent average reward: 1.4
Reward over last 100: 2.11
Average over all episodes so far: 1.9459016393442623
epsilon: 0.01
**** Episode  620 **** 
Recent average reward: 2.1
Reward over last 100: 2.14
Average over all episodes so far: 1.9483870967741936
epsilon: 0.01
**** Episode  630 **** 
Recent average reward: 1.3
Reward over last 100: 1.98
Average over all episodes so far: 1.938095238095238
epsilon: 0.01
**** Episode  640 **** 
Recent average reward: 1.5
Reward over last 100: 1.9
Average over all episodes so far: 1.93125
epsilon: 0.01
**** Episode  650 **** 
Recent average reward: 1.5
Reward over last 100: 1.76
Average over all episodes so far: 1.9246153846153846
epsilon: 0.01
**** Episode  660 **** 
Recent average reward: 2.3
Reward over last 100: 1.76
Average over all episodes so far: 1.9303030303030304
epsilon: 0.01
**** Episod

**** Episode  1180 **** 
Recent average reward: 1.2
Reward over last 100: 2.27
Average over all episodes so far: 2.1652542372881354
epsilon: 0.01
**** Episode  1190 **** 
Recent average reward: 2.5
Reward over last 100: 2.19
Average over all episodes so far: 2.168067226890756
epsilon: 0.01
**** Episode  1200 **** 
Recent average reward: 3.0
Reward over last 100: 2.16
Average over all episodes so far: 2.175
epsilon: 0.01
**** Episode  1210 **** 
Recent average reward: 2.3
Reward over last 100: 2.16
Average over all episodes so far: 2.1760330578512397
epsilon: 0.01
**** Episode  1220 **** 
Recent average reward: 3.3
Reward over last 100: 2.26
Average over all episodes so far: 2.185245901639344
epsilon: 0.01
**** Episode  1230 **** 
Recent average reward: 3.0
Reward over last 100: 2.37
Average over all episodes so far: 2.191869918699187
epsilon: 0.01
**** Episode  1240 **** 
Recent average reward: 2.7
Reward over last 100: 2.44
Average over all episodes so far: 2.1959677419354837
epsilon:

**** Episode  1750 **** 
Recent average reward: 3.2
Reward over last 100: 2.91
Average over all episodes so far: 2.352
epsilon: 0.01
**** Episode  1760 **** 
Recent average reward: 2.5
Reward over last 100: 2.75
Average over all episodes so far: 2.3528409090909093
epsilon: 0.01
**** Episode  1770 **** 
Recent average reward: 2.4
Reward over last 100: 2.77
Average over all episodes so far: 2.353107344632768
epsilon: 0.01
**** Episode  1780 **** 
Recent average reward: 2.3
Reward over last 100: 2.54
Average over all episodes so far: 2.352808988764045
epsilon: 0.01
**** Episode  1790 **** 
Recent average reward: 4.9
Reward over last 100: 2.75
Average over all episodes so far: 2.3670391061452514
epsilon: 0.01
**** Episode  1800 **** 
Recent average reward: 2.8
Reward over last 100: 2.88
Average over all episodes so far: 2.3694444444444445
epsilon: 0.01
**** Episode  1810 **** 
Recent average reward: 3.5
Reward over last 100: 2.98
Average over all episodes so far: 2.3756906077348066
epsilon

**** Episode  2320 **** 
Recent average reward: 4.4
Reward over last 100: 5.51
Average over all episodes so far: 2.5853448275862068
epsilon: 0.01
**** Episode  2330 **** 
Recent average reward: 2.5
Reward over last 100: 5.39
Average over all episodes so far: 2.584978540772532
epsilon: 0.01
**** Episode  2340 **** 
Recent average reward: 2.7
Reward over last 100: 5.42
Average over all episodes so far: 2.5854700854700856
epsilon: 0.01
**** Episode  2350 **** 
Recent average reward: 3.2
Reward over last 100: 5.24
Average over all episodes so far: 2.588085106382979
epsilon: 0.01
**** Episode  2360 **** 
Recent average reward: 3.3
Reward over last 100: 4.78
Average over all episodes so far: 2.5911016949152543
epsilon: 0.01
**** Episode  2370 **** 
Recent average reward: 3.5
Reward over last 100: 4.52
Average over all episodes so far: 2.5949367088607596
epsilon: 0.01
**** Episode  2380 **** 
Recent average reward: 2.9
Reward over last 100: 4.06
Average over all episodes so far: 2.59621848739

...

**** Episode  25710 **** 
Recent average reward: 5.7
Reward over last 100: 5.17
Average over all episodes so far: 3.915208090237262
epsilon: 0.01
**** Episode  25720 **** 
Recent average reward: 2.4
Reward over last 100: 4.87
Average over all episodes so far: 3.914618973561431
epsilon: 0.01
**** Episode  25730 **** 
Recent average reward: 3.4
Reward over last 100: 4.74
Average over all episodes so far: 3.91441896618733
epsilon: 0.01
**** Episode  25740 **** 
Recent average reward: 2.6
Reward over last 100: 4.56
Average over all episodes so far: 3.9139083139083137
epsilon: 0.01
**** Episode  25750 **** 
Recent average reward: 2.4
Reward over last 100: 4.44
Average over all episodes so far: 3.9133203883495145
epsilon: 0.01
**** Episode  25760 **** 
Recent average reward: 4.6
Reward over last 100: 4.26
Average over all episodes so far: 3.913586956521739
epsilon: 0.01
**** Episode  25770 **** 
Recent average reward: 4.1
Reward over last 100: 4.15
Average over all episodes so far: 3.9136592

**** Episode  26280 **** 
Recent average reward: 3.2
Reward over last 100: 4.03
Average over all episodes so far: 3.9116057838660576
epsilon: 0.01
**** Episode  26290 **** 
Recent average reward: 3.0
Reward over last 100: 3.78
Average over all episodes so far: 3.911259033853176
epsilon: 0.01
**** Episode  26300 **** 
Recent average reward: 3.3
Reward over last 100: 3.69
Average over all episodes so far: 3.9110266159695817
epsilon: 0.01
**** Episode  26310 **** 
Recent average reward: 4.7
Reward over last 100: 3.69
Average over all episodes so far: 3.911326491828202
epsilon: 0.01
**** Episode  26320 **** 
Recent average reward: 2.5
Reward over last 100: 3.64
Average over all episodes so far: 3.910790273556231
epsilon: 0.01
**** Episode  26330 **** 
Recent average reward: 3.2
Reward over last 100: 3.54
Average over all episodes so far: 3.910520319027725
epsilon: 0.01
**** Episode  26340 **** 
Recent average reward: 3.9
Reward over last 100: 3.56
Average over all episodes so far: 3.910516

**** Episode  26840 **** 
Recent average reward: 3.8
Reward over last 100: 3.91
Average over all episodes so far: 3.9097988077496275
epsilon: 0.01
**** Episode  26850 **** 
Recent average reward: 5.1
Reward over last 100: 4.15
Average over all episodes so far: 3.9102420856610802
epsilon: 0.01
**** Episode  26860 **** 
Recent average reward: 2.7
Reward over last 100: 3.9
Average over all episodes so far: 3.9097915115413255
epsilon: 0.01
**** Episode  26870 **** 
Recent average reward: 4.0
Reward over last 100: 3.93
Average over all episodes so far: 3.909825083736509
epsilon: 0.01
**** Episode  26880 **** 
Recent average reward: 5.9
Reward over last 100: 3.91
Average over all episodes so far: 3.910565476190476
epsilon: 0.01
**** Episode  26890 **** 
Recent average reward: 1.9
Reward over last 100: 3.72
Average over all episodes so far: 3.9098177761249535
epsilon: 0.01
**** Episode  26900 **** 
Recent average reward: 3.6
Reward over last 100: 3.6
Average over all episodes so far: 3.909702

**** Episode  27410 **** 
Recent average reward: 5.0
Reward over last 100: 3.92
Average over all episodes so far: 3.914666180226195
epsilon: 0.01
**** Episode  27420 **** 
Recent average reward: 7.3
Reward over last 100: 4.1
Average over all episodes so far: 3.915900802334063
epsilon: 0.01
**** Episode  27430 **** 
Recent average reward: 8.2
Reward over last 100: 4.32
Average over all episodes so far: 3.9174626321545754
epsilon: 0.01
**** Episode  27440 **** 
Recent average reward: 5.5
Reward over last 100: 4.5
Average over all episodes so far: 3.9180393586005833
epsilon: 0.01
**** Episode  27450 **** 
Recent average reward: 4.3
Reward over last 100: 4.72
Average over all episodes so far: 3.9181785063752277
epsilon: 0.01
**** Episode  27460 **** 
Recent average reward: 3.7
Reward over last 100: 4.69
Average over all episodes so far: 3.9180990531682447
epsilon: 0.01
**** Episode  27470 **** 
Recent average reward: 4.0
Reward over last 100: 4.82
Average over all episodes so far: 3.918128

**** Episode  27970 **** 
Recent average reward: 3.3
Reward over last 100: 3.66
Average over all episodes so far: 3.9226313907758312
epsilon: 0.01
**** Episode  27980 **** 
Recent average reward: 2.1
Reward over last 100: 3.43
Average over all episodes so far: 3.9219799857040742
epsilon: 0.01
**** Episode  27990 **** 
Recent average reward: 5.6
Reward over last 100: 3.61
Average over all episodes so far: 3.922579492675956
epsilon: 0.01
**** Episode  28000 **** 
Recent average reward: 4.0
Reward over last 100: 3.59
Average over all episodes so far: 3.9226071428571427
epsilon: 0.01
**** Episode  28010 **** 
Recent average reward: 2.7
Reward over last 100: 3.51
Average over all episodes so far: 3.9221706533380933
epsilon: 0.01
**** Episode  28020 **** 
Recent average reward: 3.2
Reward over last 100: 3.55
Average over all episodes so far: 3.921912919343326
epsilon: 0.01
**** Episode  28030 **** 
Recent average reward: 5.9
Reward over last 100: 3.58
Average over all episodes so far: 3.9226

In [ ]:
plt.figure(2,figsize=[12,5])
plt.title("Score at end of episode")
plt.plot(reward_total[:frames_total])
plt.show()

In [ ]:
print("Average number of steps: {}". format(np.average(steps_total[:frames_total])))

In [ ]:
print("Average number of steps=reward: {}". format(np.average(steps_total)))
print("Average reward in last 100 episodes: {}". format(np.average(reward_total[frames_total-100:frames_total])))



plt.figure(1,figsize=[12,5])
plt.title("Rewards")
plt.bar(torch.arange(len(rewards_total)), rewards_total,alpha=0.6, color='green')
#plt.plot(rewards_total)
plt.show()

In [ ]:
plt.figure(2,figsize=[12,5])
plt.title("Steps to finish episode")
plt.plot(steps_total[:frames_total])
plt.show()